In [1]:
!pip install 'fastai>=1.0.6'
!pip install ../input/spacy_model_en/en_core_web_sm-2.0.0.tar.gz
!python -m spacy link en_core_web_sm en
from fastai import *
from fastai.text import *
from tensorboard_cb import *

    100% |████████████████████████████████| 102kB 4.1MB/s 
    100% |████████████████████████████████| 23.3MB 2.8MB/s 
    100% |████████████████████████████████| 1.9MB 22.3MB/s 
    100% |████████████████████████████████| 92kB 32.1MB/s 
spacy 2.0.16 has requirement regex==2018.01.10, but you'll have regex 2017.4.5 which is incompatible.
mkl-devel 2018.0.3 has requirement mkl==2018.0.3, but you'll have mkl 2019.0 which is incompatible.
menpo 0.8.1 has requirement matplotlib<2.0,>=1.4, but you'll have matplotlib 2.2.3 which is incompatible.
menpo 0.8.1 has requirement pillow<5.0,>=3.0, but you'll have pillow 5.2.0 which is incompatible.
menpo 0.8.1 has requirement scipy<1.0,>=0.16, but you'll have scipy 1.1.0 which is incompatible.
  Found existing installation: fastprogress 0.1.9
    Uninstalling fastprogress-0.1.9:
      Successfully uninstalled fastprogress-0.1.9
  Found existing installation: cymem 1.31.2
    Uninstalling cymem-1.31.2:
      Successfully uninstalled cymem-1.31.2
  F

# Text example

An example of creating a language model and then transfering to a classifier.

In [2]:
def read_csv_with_sample_size(path, sample_size=10, chunksize=24000):
  total_rows = get_total_length(path, chunksize=chunksize)
  frac = sample_size/total_rows
  df = pd.DataFrame()
  for chunk in pd.read_csv(path, chunksize=chunksize, header=None):
    df = pd.concat([df,chunk.sample(frac=frac)])
  return df

In [3]:
!rm -rf data/
def build_directory_from_mounts(src='/floyd/input/imdb_reviews_wt103/', dst='data/', sample_size=1000):
    """
    Copy mounted directory into writable local, then save training language model csv with the specified sample_size
    """
    shutil.copytree(src, dst)
    dftr_lm = read_csv_with_sample_size(
        dst+'train.csv', sample_size=sample_size)
    dftr_lm.to_csv(dst+'train_lm.csv', index=False)
build_directory_from_mounts(sample_size=100)

In [4]:
dir_path = 'data/'
dir_path

'data/'

In [5]:
dftr_lm = pd.read_csv(dir_path+'train_lm.csv', header=None)
dftr_clas = pd.read_csv(dir_path+'train_clas.csv', header=None)
dftr_lm.head()

,0,1
0,0,1
1,0,This story is a complex and wonderful tale of ...
2,0,"This is a pretty disappointing movie, coming o..."
3,0,"First off, the less you know about this movie ..."
4,0,"The story is seen before, but that does'n matt..."


In [6]:
dftr_clas.head()

,0,1
0,0,The people who are praising this film are the ...
1,1,A kind of road movie in old-fashioned trains i...
2,0,We rented five movies for New Year's Eve weeke...
3,1,"Ok, basically this is a popcorn sci-fi movie, ..."
4,0,"I haven't seen ""Henry Fool"", but after watchin..."


Open and view the independent and dependent variables:

In [7]:
len(dftr_lm)

101

In [8]:
classes = read_classes(dir_path+'/classes.txt')[:2]
classes

['neg', 'pos']

Create a `DataBunch` for each of the language model and the classifier:

In [9]:
data_lm = TextLMDataBunch.from_csv(dir_path, train="train_lm")
data_clas = TextClasDataBunch.from_csv(dir_path, train='train_clas', valid='valid_clas', vocab=data_lm.train_ds.vocab)

Tokenizing train_lm.


Numericalizing train_lm.
Tokenizing valid.


Numericalizing valid.
Tokenizing train_clas.


Numericalizing train_clas.
Tokenizing valid_clas.


Numericalizing valid_clas.


We'll fine-tune the language model:

In [11]:
wd = 1e-7
lr=1e-3
lrs = lr
learn = RNNLearner.language_model(data_lm,drop_mult=0.7, pretrained_fnames=['lstm_wt103', 'itos_wt103'])
learn.fit_one_cycle(max_lr=slice(1e-3/2,1), wd=wd, cyc_len=1)

Total time: 03:18
epoch  train loss  valid loss  accuracy
1      9.945601    9.194844    0.062284  (03:18)



Save our language model's encoder:

In [ ]:
learn.save_encoder(f'lm_last_ft')
learn.load_encoder(f'lm_last_ft')
learn.unfreeze()

In [12]:
learn.lr_find(start_lr=lrs/10, end_lr=lrs*10)

In [13]:
learn.fit(lr = slice(lrs, 1), wd=wd, epochs=15)

KeyboardInterrupt: 

In [15]:
learn.save_encoder('lm1_enc')

Fine tune it to create a classifier:

In [16]:
learn = RNNLearner.classifier(data_clas, drop_mult=0.5, clip=0.25)
learn.metrics = [accuracy]
learn.load_encoder('lm1_enc')

In [ ]:
wd = 1e-7
learn.load_encoder('lm1_enc')
learn.freeze_to(-1)
learn.lr_find(lrs/1000)
learn.fit(lr=slice(1e-4, 1e-2), wd=wd, epochs=1)
learn.save('clas_0')
learn.load('clas_0')
learn.freeze_to(-2)
learn.fit(lr=slice(1e-4, 1e-2), wd=wd, epochs=1)
learn.save('clas_1')
learn.load('clas_1')
learn.unfreeze()
learn.fit(lr=slice(1e-4, 1e-2), wd=wd, epochs=14)
learn.save('clas_2')